In [27]:


!pip install transformers torch pandas scikit-learn


In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import pandas as pd


df = pd.read_csv('/content/drive/My Drive/BERT2/training_data.csv')
df = df.sample(frac=0.10, random_state=42)


In [54]:
print(df.columns)

Index(['News Article Topic', 'News Article Topic Number'], dtype='object')


In [55]:
topic_mapping = {0: 'World', 1: 'sports', 2: 'Business', 3: 'Sci/Tech'}
df['News Article Topic'] = df['News Article Topic Number'].map(topic_mapping)


In [58]:

test_df = pd.read_csv('/content/drive/My Drive/BERT2/sentiment-topic-test.tsv', delimiter='\t')
print(test_df.columns)


X_test = test_df['text'].values
y_test = test_df['topic'].values


Index(['sentence id', 'text', 'sentiment', 'topic'], dtype='object')


In [59]:
X_train = df['News Article Topic'].values
y_train = df['News Article Topic Number'].values


In [60]:
!pip install transformers

In [61]:
from transformers import RobertaTokenizer
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [62]:


train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)


In [63]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings.input_ids)


train_dataset = NewsDataset(train_encodings, y_train)
test_dataset = NewsDataset(test_encodings, y_test)


In [40]:
!pip install transformers
!pip install accelerate

In [41]:
#Stupid workaround for program not recognizing accelerate
!pip uninstall accelerate
!pip install -i https://testpypi.python.org/pypi accelerate
!accelerate env
!accelerate test

Found existing installation: accelerate 0.28.0
Uninstalling accelerate-0.28.0:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-estimate-memory
    /usr/local/bin/accelerate-launch
    /usr/local/lib/python3.10/dist-packages/accelerate-0.28.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/accelerate/*
Proceed (Y/n)? y
  Successfully uninstalled accelerate-0.28.0
Looking in indexes: https://testpypi.python.org/pypi
  Using cached https://test-files.pythonhosted.org/packages/a0/11/9bfcf765e71a2c84bbf715719ba520aeacb2ad84113f14803ff1947ddf69/accelerate-0.28.0-py3-none-any.whl (290 kB)



Copy-and-paste the text below in your GitHub issue

- `Accelerate` version: 0.28.0
- Platform: Linux-6.1.58+-x86_64-with-glibc2.35
- Python version: 3.10.12
- Numpy version: 1.25.2
- PyTorch version (GPU?): 2.2.1+cu121 (True)
- PyTorch XPU available: False
- PyTorch NPU available: False
- System RAM: 50.99 GB
- GPU type: Tesla V100-SXM2-16GB
- `Accelerate` default config:
	Not found

Running:  accelerate-launch /usr/local/lib/python3.10/dist-packages/accelerate/test_utils/scripts/test_script.py
stderr: The following values were not passed to `accelerate launch` and had defaults used instead:
stderr: 	`--num_processes` was set to a value of `1`
stderr: 	`--num_machines` was set to a value of `1`
stderr: 	`--mixed_precision` was set to a value of `'no'`
stderr: 	`--dynamo_backend` was set to a value of `'no'`
stderr: To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
stdout: **Initialization**
stdout: Testing, testing. 1, 2, 3.
stdout

In [64]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=564, training_loss=0.15246497460885355, metrics={'train_runtime': 41.3146, 'train_samples_per_second': 871.362, 'train_steps_per_second': 13.651, 'total_flos': 111001969728000.0, 'train_loss': 0.15246497460885355, 'epoch': 3.0})

In [65]:
!pip install datasets
from datasets import load_dataset



In [ ]:

#test_df = pd.read_csv('/content/drive/My Drive/BERT2/sentiment-topic-test.tsv', delimiter='\t')

#X_test = test_df['text'].values
#y_test = test_df['topic'].values

In [66]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)




In [67]:
from transformers import AutoTokenizer

model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


X_test_tokenized = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors="pt")


In [68]:
from torch.utils.data import DataLoader, Dataset


class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


test_dataset = TestDataset(X_test_tokenized, y_test_encoded)

test_loader = DataLoader(test_dataset, batch_size=48, shuffle=True)


In [69]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# from torch.utils.data import DataLoader


# test_loader = DataLoader(test_dataset, batch_size=48, shuffle=True)  # Adjust batch_size to your preference


In [70]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].to(device)
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Accuracy: {accuracy}")


Accuracy: 0.4


<ipython-input-68-833411069f00>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [71]:

true_labels = []
predictions = []

model.eval()
with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        output = model(**inputs)
        logits = output.logits
        batch_predictions = torch.argmax(logits, dim=1)
        predictions.extend(batch_predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())


<ipython-input-68-833411069f00>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [72]:
from sklearn.metrics import classification_report


report = classification_report(true_labels, predictions, target_names=label_encoder.classes_)

print(report)


              precision    recall  f1-score   support

        book       0.00      0.00      0.00         3
       movie       0.40      1.00      0.57         4
      sports       0.00      0.00      0.00         3

    accuracy                           0.40        10
   macro avg       0.13      0.33      0.19        10
weighted avg       0.16      0.40      0.23        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
